In [1]:
import pandas as pd
data = pd.read_csv('data.csv')
%store data

Stored 'data' (DataFrame)


In [2]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/skynet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/skynet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skynet/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm


def preprocess_text(messages):
    corpus = []
    lemmatizer = WordNetLemmatizer()

    # Use tqdm to add a progress bar to the loop
    for sentence in tqdm(messages["review"], desc="Processing reviews"):
        sentence = re.sub("[^a-zA-Z]", " ", sentence.lower())
        words = word_tokenize(sentence)
        lemmatized_words = [
            lemmatizer.lemmatize(word)
            for word in words
            if word not in set(stopwords.words("english"))
        ]
        lemmatized_review = " ".join(lemmatized_words)
        corpus.append(lemmatized_review)

    return corpus



corpus = preprocess_text(data)
corpus[1:11]

Processing reviews: 100%|██████████| 50000/50000 [20:11<00:00, 41.28it/s]


['wonderful little production br br filming technique unassuming old time bbc fashion give comforting sometimes discomforting sense realism entire piece br br actor extremely well chosen michael sheen got polari voice pat truly see seamless editing guided reference williams diary entry well worth watching terrificly written performed piece masterful production one great master comedy life br br realism really come home little thing fantasy guard rather use traditional dream technique remains solid disappears play knowledge sens particularly scene concerning orton halliwell set particularly flat halliwell mural decorating every surface terribly well done',
 'thought wonderful way spend time hot summer weekend sitting air conditioned theater watching light hearted comedy plot simplistic dialogue witty character likable even well bread suspected serial killer may disappointed realize match point risk addiction thought proof woody allen still fully control style many u grown love br br lau

In [5]:
# from nltk import sent_tokenize
# from nltk import WordNetLemmatizer
# from gensim.utils import simple_preprocess

# words = []
# for _ in corpus:
#     sent_token = sent_tokenize(_)
#     for __ in sent_token:
#         words.append(simple_preprocess(__))

In [6]:
import gensim
# models = gensim.models.Word2Vec(
#     words, vector_size=300, window=21, min_count=2, epochs=100
# )
# loading the model
models = gensim.models.Word2Vec.load("word2vec.model")

In [7]:
# get the vocabulary
vocabulary = models.wv.key_to_index

In [8]:
len(vocabulary)

54969

In [9]:
# getting the most similar words
models.wv.most_similar("good")

[('great', 0.6357738971710205),
 ('bad', 0.5855804085731506),
 ('nice', 0.5798599123954773),
 ('really', 0.5667163729667664),
 ('decent', 0.5639677047729492),
 ('liked', 0.5442432165145874),
 ('ok', 0.5278295874595642),
 ('better', 0.47997018694877625),
 ('alright', 0.4648732542991638),
 ('lot', 0.46387770771980286)]

In [10]:
import numpy as np
from tqdm import tqdm


def avg_word2vec_with_progress(corpus, models):
    avg_word2vec = []

    # Use tqdm to add a progress bar to the loop
    for sentence in tqdm(corpus, desc="Calculating Average Word2Vec"):
        vector = np.zeros(300)
        count = 0
        for word in sentence.split():
            if word in models.wv.index_to_key:
                vector += models.wv[word]
                count += 1
        if count != 0:
            vector /= count
        avg_word2vec.append(vector)

    return avg_word2vec

avg_word2vec_result = avg_word2vec_with_progress(corpus, models)

Calculating Average Word2Vec: 100%|██████████| 50000/50000 [03:32<00:00, 234.95it/s]


In [11]:
y = data["sentiment"]


In [13]:
from sklearnex import patch_sklearn
patch_sklearn()
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm


def apply_ml_algorithms(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Label encoding for the target variable
    le = LabelEncoder()
    y_train_encoded = le.fit_transform(y_train)
    y_test_encoded = le.transform(y_test)

    # Define machine learning models
    models = {
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "Support Vector Machine": SVC(kernel="linear", C=1.0, random_state=42),
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    }

    results = {}

    # Apply each machine learning algorithm
    for model_name, model in tqdm(models.items(), desc="Applying ML Algorithms"):
        model.fit(X_train, y_train_encoded)
        predictions = model.predict(X_test)

        # Evaluate the model
        accuracy = accuracy_score(y_test_encoded, predictions)
        report = classification_report(y_test_encoded, predictions, output_dict=True)

        results[model_name] = {"accuracy": accuracy, "classification_report": report}

    return results


# Example usage:
# Assuming avg_word2vec_result is your feature dataset and y is your target variable
results = apply_ml_algorithms(avg_word2vec_result, y)

# Print results
for model_name, result in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {result['accuracy']:.4f}")
    print("Classification Report:\n", result["classification_report"])
    print("-" * 50)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
Applying ML Algorithms: 100%|██████████| 3/3 [00:33<00:00, 11.13s/it]

Model: Random Forest
Accuracy: 0.8613
Classification Report:
 {'0': {'precision': 0.8760521885521886, 'recall': 0.8391453336020963, 'f1-score': 0.8572016884587665, 'support': 4961.0}, '1': {'precision': 0.8479420731707317, 'recall': 0.883111728517563, 'f1-score': 0.8651696315738311, 'support': 5039.0}, 'accuracy': 0.8613, 'macro avg': {'precision': 0.8619971308614601, 'recall': 0.8611285310598297, 'f1-score': 0.8611856600162988, 'support': 10000.0}, 'weighted avg': {'precision': 0.8618875014114725, 'recall': 0.8613, 'f1-score': 0.8612167349944475, 'support': 10000.0}}
--------------------------------------------------
Model: Support Vector Machine
Accuracy: 0.8820
Classification Report:
 {'0': {'precision': 0.8858950806286997, 'recall': 0.8748236242693005, 'f1-score': 0.8803245436105477, 'support': 4961.0}, '1': {'precision': 0.8782591648696334, 'recall': 0.8890652907322881, 'f1-score': 0.8836291913214991, 'support': 5039.0}, 'accuracy': 0.882, 'macro avg': {'precision': 0.882077122749